# IMPORTING MODULES

In [108]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import numpy

# MY FIRST MODEL

In [109]:
model = nn.Sequential(
    nn.Linear(28*28,64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,28),
    nn.ReLU(),
    nn.Linear(28,10)
)

# OPTIMIZER 🚀

In [110]:
optimiser = optim.SGD(model.parameters(), lr = 1e-2)

# DEFINING LOSS 

In [111]:
loss = nn.CrossEntropyLoss()

# SPLITTING THE DATA

In [112]:
train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
train,dev = random_split(train_data,[55000,5000])
train_loader = DataLoader(train,batch_size=32)
dev_loader = DataLoader(dev,batch_size=32)

# EPOCHS || LOOP

In [113]:
def propagate(loader):
    losses = list()
    for batch in loader:
        x,y = batch
        x = x.view(x.size(0), -1)
        logit = model(x)
        J = loss(logit,y)
        model.zero_grad()
        J.backward()
        optimiser.step()
        losses.append(J.item())
    return losses
    
epochs = 10
for epoch in range(epochs):
    print(f'@ Epoch = {epoch+1}\nTraining Loss = {torch.tensor(propagate(train_loader)).mean():.2f}')
    print(f'Validation Loss = {torch.tensor(propagate(dev_loader)).mean():.2f}\n--------------------------')
    

@ Epoch = 1
Training Loss = 2.28
Validation Loss = 2.06
--------------------------
@ Epoch = 2
Training Loss = 1.09
Validation Loss = 0.63
--------------------------
@ Epoch = 3
Training Loss = 0.51
Validation Loss = 0.42
--------------------------
@ Epoch = 4
Training Loss = 0.37
Validation Loss = 0.32
--------------------------
@ Epoch = 5
Training Loss = 0.29
Validation Loss = 0.25
--------------------------
@ Epoch = 6
Training Loss = 0.24
Validation Loss = 0.20
--------------------------
@ Epoch = 7
Training Loss = 0.20
Validation Loss = 0.17
--------------------------
@ Epoch = 8
Training Loss = 0.17
Validation Loss = 0.15
--------------------------
@ Epoch = 9
Training Loss = 0.15
Validation Loss = 0.13
--------------------------
@ Epoch = 10
Training Loss = 0.13
Validation Loss = 0.11
--------------------------


In [55]:
torch.save(model,"models/mnist.pt")

In [92]:
model = torch.load('models/mnist.pt')

In [93]:
test_set = datasets.MNIST('data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_set, batch_size=100, shuffle=True)

In [107]:
correct = 0 ; total = 0
with torch.no_grad():
    for data in test_loader:
        x, y = data
        output = model(x.view(-1, 784))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct +=1
            total +=1
print(f'Accuracy: {round(correct/total, 3)}')

TypeError: 'str' object is not callable

In [95]:
def digit(th):
    return str(int(torch.argmax(model(x[th].view(-1, 784))[0])))

In [96]:
import cv2
th=0
while True:
    frame = numpy.array(transforms.ToPILImage()(x[th]).convert("RGB"))[:, :, ::-1].copy()
    frame = cv2.resize(frame,(900,900))
    cv2.putText(frame, 'DIGIT : '+str(digit(th)), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 5)
    cv2.imshow('DIGIT',frame)
    k = cv2.waitKey()
    if k==27:
        break
    elif k==110 and th<15:
        th+=1
    elif k==112 and th>0:
        th-=1
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x55f50a57f040) is not the object's thread (0x55f50a587a10).
Cannot move to target thread (0x55f50a57f040)

QObject::moveToThread: Current thread (0x55f50a57f040) is not the object's thread (0x55f50a587a10).
Cannot move to target thread (0x55f50a57f040)

QObject::moveToThread: Current thread (0x55f50a57f040) is not the object's thread (0x55f50a587a10).
Cannot move to target thread (0x55f50a57f040)

QObject::moveToThread: Current thread (0x55f50a57f040) is not the object's thread (0x55f50a587a10).
Cannot move to target thread (0x55f50a57f040)

QObject::moveToThread: Current thread (0x55f50a57f040) is not the object's thread (0x55f50a587a10).
Cannot move to target thread (0x55f50a57f040)

QObject::moveToThread: Current thread (0x55f50a57f040) is not the object's thread (0x55f50a587a10).
Cannot move to target thread (0x55f50a57f040)

QObject::moveToThread: Current thread (0x55f50a57f040) is not the object's thread (0x55f50a587a10).
Cannot move to tar

In [105]:
torch.onnx.export(model,
                 torch.randn(1, 1,28,28),
                 "models/mnist.onnx",
                 verbose=False,
                 input_names=[ "INPUTS" ],
                 output_names=[ "OUTPUTS" ],
                 export_params=True)

AttributeError: 'str' object has no attribute 'modules'

# INFERENCING

In [103]:
import onnxruntime
import numpy as np

def infer(img):
    result = session.run([output_name], {input_name: img.numpy()})
    prediction=int(np.argmax(np.array(result).squeeze()))
    print("PREDICTED : "+str(prediction),end=" ********* ")
    
model_dir ="./models"
model=model_dir+"/mnist.onnx"
session = onnxruntime.InferenceSession(model)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
sample = next(iter(test_loader))
imgs, lbls = sample
for x in range(10):
    infer(imgs[x:x+1])
    print("LABEL : "+str(int(lbls[x])))

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: INPUTS for the following indices
 index: 2 Got: 28 Expected: 64
 index: 3 Got: 28 Expected: 784
 Please fix either the inputs or the model.